In [1]:
import os
import mysql.connector as connector
import logging
from time import time

In [2]:
logger = logging.getLogger("[Relational-Database-Administration MySQL]")
if os.path.exists("../log/rda-fxn-mysql.log"):
  os.remove("../log/rda-fxn-mysql.log")
logging.basicConfig(filename='../log/rda-fxn-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

In [3]:
logger.info("Creating a connection between MySQL and Python")
dbconfig={"user":"user_wsl", "password":os.environ["MYSQL_USER_WSL_PASSWORD"], "port":33061, "host":"localhost"}
connection=connector.connect(**dbconfig)
print("Connection established between MySQL and Python")
logger.info("Connection established between MySQL and Python")

Connection established between MySQL and Python


In [4]:
print("Creating cursor object from the connection")
logger.info("Creating cursor object from the connection")
cursor = connection.cursor(buffered=True)
print("Cursor object created to communicate with MySQL using Python.")
logger.info("Cursor object created to communicate with MySQL using Python.")

Creating cursor object from the connection
Cursor object created to communicate with MySQL using Python.


In [5]:
def select_all_query(table_name: str):
    return f"""SELECT * FROM {table_name};"""


def display_results(table_column_names: list, results: list, exec_time):
    #start = timer()
    table_columns_length = [len(x) for x in table_column_names]
    for result in results:
        for value in range(len(result)):
            row_data = result[value]
            if row_data:
                row_data = str(row_data)
                length_row_data = len(row_data)
                if length_row_data > table_columns_length[value]:
                    if length_row_data > 60:
                        length_row_data = length_row_data + 10
                    table_columns_length[value] = length_row_data
    dashes_plus = ""
    for num in range(len(table_columns_length)):
        dashes_plus = dashes_plus + "+" + '-'*(table_columns_length[num]+2)
    dashes_plus = dashes_plus + "+"
    
    print(dashes_plus)
    
    table_headers = ""
    for num in range(len(table_column_names)):
        table_headers = table_headers + f"| {table_column_names[num]:^{table_columns_length[num]}} "
    table_headers = table_headers + "|"
    print(table_headers)
    
    print(dashes_plus)
    
    for result in results:
        table_row = ""
        for value in range(len(result)):
            row_data = result[value]
            if row_data is None:
                row_data = "NULL"            
            table_row = table_row + "|" + f"{str(row_data):^{table_columns_length[value]+2}}"
        print(table_row + "|")
    print(dashes_plus)
    num_rows: int = len(results)
    message: str = "row returned" if num_rows == 1 else "rows returned"
    print(f"{num_rows} {message} in set: ({exec_time} sec)")

def execute_display_query_results(query: str = "", table_column_names: list = [], results: list = []): 
    logger.info(f"Executing the query: {query}")
    if len(query) > 2 and (table_column_names or results):
        print("You can only pass in the query alone or the table_column_names and results list")
        assert False
    if query and not table_column_names and not results:
        init_time = time()
        cursor.execute(query)
        end_time = time()
        exec_time = end_time - init_time
        results = cursor.fetchall()    
        table_column_names = cursor.column_names
        
    
    display_results(table_column_names, results, round(exec_time, 4))
   

In [6]:
show_version_query = """SHOW DATABASES;"""
execute_display_query_results(query=show_version_query)

+------------------------+
|        Database        |
+------------------------+
|          CVD           |
|     LittleLemonDB      |
|    STAFF_LOCATIONS     |
|        billing         |
|     classicmodels      |
|      db_Exercise       |
|         db_hr          |
|       db_learner       |
|    db_little_lemon     |
|        db_meta         |
| db_meta_advanced_mysql |
|   db_meta_dsm_mysql    |
|        db_music        |
|     db_mysqladmin      |
|     db_subqueries      |
|        db_views        |
|     db_window_fxn      |
|       employees        |
|   global_super_store   |
|   information_schema   |
|      little_lemon      |
|    little_lemon_db     |
|      lucky_Shrub       |
|     mangata_gallo      |
|     mangata_jw_db      |
|        meta_db         |
|       mg_schema        |
|         mysql          |
|   performance_schema   |
|         sakila         |
|          sys           |
|         world          |
+------------------------+
32 rows returned in set: (0.

In [7]:
show_query = """SELECT schema_name
FROM information_schema.schemata;"""
execute_display_query_results(query=show_query)

+------------------------+
|      SCHEMA_NAME       |
+------------------------+
|         mysql          |
|   information_schema   |
|   performance_schema   |
|          sys           |
|       db_learner       |
|          CVD           |
|         db_hr          |
|    STAFF_LOCATIONS     |
|        db_meta         |
|      lucky_Shrub       |
|      db_Exercise       |
|         sakila         |
|     db_mysqladmin      |
|    db_little_lemon     |
|      little_lemon      |
|    little_lemon_db     |
|        meta_db         |
|       mg_schema        |
|     mangata_gallo      |
|     mangata_jw_db      |
|     LittleLemonDB      |
|   global_super_store   |
|     classicmodels      |
|   db_meta_dsm_mysql    |
| db_meta_advanced_mysql |
|     db_subqueries      |
|        db_views        |
|        db_music        |
|       employees        |
|         world          |
|        billing         |
|     db_window_fxn      |
+------------------------+
32 rows returned in set: (0.

## Task 2.2.1: Restore MySQL server using a previous backup. 

In [8]:
use_database_query = "USE billing;"
cursor.execute(use_database_query)

In [9]:
show_tables_query = """SHOW TABLES;"""
execute_display_query_results(query=show_tables_query)

+-------------------+
| Tables_in_billing |
+-------------------+
|     billdata      |
+-------------------+
1 row returned in set: (0.0117 sec)


## Task 2.2.2 - Find the table data size

In [10]:
select_query = """select table_name, data_length + index_length from information_schema.tables where table_name = 'billdata';"""
execute_display_query_results(select_query)

+------------+----------------------------+
| TABLE_NAME | data_length + index_length |
+------------+----------------------------+
|  billdata  |          6832128           |
+------------+----------------------------+
1 row returned in set: (0.0011 sec)


In [11]:
select_query = """select table_name, (data_length + index_length)/1024 from information_schema.tables where table_name = 'billdata';"""
execute_display_query_results(select_query)

+------------+-----------------------------------+
| TABLE_NAME | (data_length + index_length)/1024 |
+------------+-----------------------------------+
|  billdata  |             6672.0000             |
+------------+-----------------------------------+
1 row returned in set: (0.0014 sec)


## Task 2.3.1 - Baseline query performance
Write a query to select all rows with a billedamount > 19999 in table billdata.

In [12]:
select_query = """select * from billdata where billedamount > 19999;"""
execute_display_query_results(select_query)

+--------+------------+--------------+---------+
| billid | customerid | billedamount | monthid |
+--------+------------+--------------+---------+
|  8509  |    285     |    20000     |  20096  |
| 68268  |    559     |    20000     |  20146  |
| 81622  |    643     |    20000     |  20157  |
| 84858  |    317     |    20000     |  20161  |
| 89353  |    871     |    20000     |  20163  |
| 102682 |    937     |    20000     |  20174  |
| 109574 |    386     |    20000     | 201810  |
| 121844 |    777     |    20000     | 201910  |
+--------+------------+--------------+---------+
8 rows returned in set: (0.0011 sec)


## Task 2.3.2: Create an index. 

In [13]:
drop_index_query = """ALTER TABLE billdata DROP INDEX idx_billedamount;"""
cursor.execute(drop_index_query)
create_index_query = """CREATE INDEX idx_billedamount on billdata(billedamount);"""
cursor.execute(create_index_query)

## Task 2.3.3: Document the improvement in query performance

In [14]:
select_query = """select * from billdata where billedamount > 19999;"""
execute_display_query_results(select_query)

+--------+------------+--------------+---------+
| billid | customerid | billedamount | monthid |
+--------+------------+--------------+---------+
|  8509  |    285     |    20000     |  20096  |
| 68268  |    559     |    20000     |  20146  |
| 81622  |    643     |    20000     |  20157  |
| 84858  |    317     |    20000     |  20161  |
| 89353  |    871     |    20000     |  20163  |
| 102682 |    937     |    20000     |  20174  |
| 109574 |    386     |    20000     | 201810  |
| 121844 |    777     |    20000     | 201910  |
+--------+------------+--------------+---------+
8 rows returned in set: (0.001 sec)


## Task 2.4.1: Find Supported Storage Engines

In [15]:
show_engines_query = """SHOW ENGINES;"""
execute_display_query_results(query=show_engines_query)

+--------------------+---------+--------------------------------------------------------------------------+--------------+-----+------------+
|       Engine       | Support |                                 Comment                                  | Transactions | XA  | Savepoints |
+--------------------+---------+--------------------------------------------------------------------------+--------------+-----+------------+
|      ARCHIVE       |   YES   |                          Archive storage engine                          |      NO      | NO  |     NO     |
|     BLACKHOLE      |   YES   |      /dev/null storage engine (anything you write to it disappears)      |      NO      | NO  |     NO     |
|     MRG_MYISAM     |   YES   |                  Collection of identical MyISAM tables                   |      NO      | NO  |     NO     |
|     FEDERATED      |   NO    |                      Federated MySQL storage engine                      |     NULL     |NULL |    NULL    |
|     

## Task 2.4.2: Find Supported Storage Engine of a table

In [16]:
show_table_engine_query = """SELECT TABLE_NAME, ENGINE FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'billdata';"""
execute_display_query_results(query=show_table_engine_query)

+------------+--------+
| TABLE_NAME | ENGINE |
+------------+--------+
|  billdata  | InnoDB |
+------------+--------+
1 row returned in set: (0.001 sec)


In [17]:
use_database_query = "USE information_schema;"
cursor.execute(use_database_query)

show_tables_query = """SHOW TABLES;"""
execute_display_query_results(query=show_tables_query)

+---------------------------------------+
|     Tables_in_information_schema      |
+---------------------------------------+
|   ADMINISTRABLE_ROLE_AUTHORIZATIONS   |
|           APPLICABLE_ROLES            |
|            CHARACTER_SETS             |
|           CHECK_CONSTRAINTS           |
|              COLLATIONS               |
| COLLATION_CHARACTER_SET_APPLICABILITY |
|                COLUMNS                |
|          COLUMNS_EXTENSIONS           |
|           COLUMN_PRIVILEGES           |
|           COLUMN_STATISTICS           |
|             ENABLED_ROLES             |
|                ENGINES                |
|                EVENTS                 |
|                 FILES                 |
|          INNODB_BUFFER_PAGE           |
|        INNODB_BUFFER_PAGE_LRU         |
|       INNODB_BUFFER_POOL_STATS        |
|         INNODB_CACHED_INDEXES         |
|              INNODB_CMP               |
|             INNODB_CMPMEM             |
|          INNODB_CMPMEM_RESET    

In [20]:
show_tables_query = """SHOW COLUMNS from columns;"""
execute_display_query_results(query=show_tables_query)

+--------------------------+----------------------------+------+-----+---------+-------+
|          Field           |            Type            | Null | Key | Default | Extra |
+--------------------------+----------------------------+------+-----+---------+-------+
|      TABLE_CATALOG       |        varchar(64)         |  NO  |     |  NULL   |       |
|       TABLE_SCHEMA       |        varchar(64)         |  NO  |     |  NULL   |       |
|        TABLE_NAME        |        varchar(64)         |  NO  |     |  NULL   |       |
|       COLUMN_NAME        |        varchar(64)         | YES  |     |  NULL   |       |
|     ORDINAL_POSITION     |        int unsigned        |  NO  |     |  NULL   |       |
|      COLUMN_DEFAULT      |            text            | YES  |     |  NULL   |       |
|       IS_NULLABLE        |         varchar(3)         |  NO  |     |         |       |
|        DATA_TYPE         |          longtext          | YES  |     |  NULL   |       |
| CHARACTER_MAXIMUM_L

In [21]:
select_query = """SELECT COLUMN_NAME FROM COLUMNS WHERE TABLE_NAME = 'country';"""
execute_display_query_results(query=select_query)

+----------------+
|  COLUMN_NAME   |
+----------------+
|   country_id   |
|    country     |
|  last_update   |
|      Code      |
|      Name      |
|   Continent    |
|     Region     |
|  SurfaceArea   |
|   IndepYear    |
|   Population   |
| LifeExpectancy |
|      GNP       |
|     GNPOld     |
|   LocalName    |
| GovernmentForm |
|  HeadOfState   |
|    Capital     |
|     Code2      |
+----------------+
18 rows returned in set: (0.0017 sec)


In [22]:
select_query = """SELECT DISTINCT TABLE_NAME FROM COLUMNS WHERE TABLE_SCHEMA = 'db_learner';"""
execute_display_query_results(query=select_query)

+--------------+
|  TABLE_NAME  |
+--------------+
| BankAccounts |
|  PETRESCUE   |
|   PETSALE    |
|   ShoeShop   |
+--------------+
4 rows returned in set: (0.0016 sec)


In [23]:
select_query = """SELECT DISTINCT TABLE_NAME FROM COLUMNS WHERE TABLE_SCHEMA = 'db_meta_advanced_mysql';"""
execute_display_query_results(query=select_query)

+-------------------+
|    TABLE_NAME     |
+-------------------+
|     customers     |
|   tbl_customers   |
| tbl_notifications |
|    tbl_orders     |
|   tbl_products    |
+-------------------+
5 rows returned in set: (0.0014 sec)


In [24]:
select_query = """SELECT table_name, (data_length + index_length)/1024 FROM INFORMATION_SCHEMA.TABLES 
WHERE table_name = 'country' OR table_name = 'city' 
OR table_name = 'countrylanguage' OR table_name = 'csv_test';"""

execute_display_query_results(query=select_query)

+-----------------+-----------------------------------+
|   TABLE_NAME    | (data_length + index_length)/1024 |
+-----------------+-----------------------------------+
|      city       |              64.0000              |
|     country     |              16.0000              |
|      city       |             496.0000              |
|     country     |              96.0000              |
| countrylanguage |             160.0000              |
+-----------------+-----------------------------------+
5 rows returned in set: (0.0025 sec)


In [25]:
show_databases_query = """SHOW DATABASES;"""
cursor.execute(show_databases_query)
databases = cursor.fetchall()
for database in databases:
    show_tables_query = f"""SHOW TABLES FROM {database[0]}"""
    cursor.execute(show_tables_query)
    tables = cursor.fetchall()
    print(f"{database[0]}:")
    for table in tables:
        print(table[0], end = ",")
    print("\n")

CVD:
MEDICAL_DEPARTMENTS,MEDICAL_HISTORY,MEDICAL_LOCATIONS,MEDICAL_PROCEDURES,PATIENTS,

LittleLemonDB:
Bookings,Customers,Menu,MenuItems,OrderDelivery,Orders,OrdersView,Staff,

STAFF_LOCATIONS:
staff_locations_view,

billing:
billdata,

classicmodels:
customers,employees,offices,orderdetails,orders,payments,productlines,products,

db_Exercise:
Bookings,Customers,FoodOrders,LowCostMenuItems,MenuItems,Menus,Staff,TableOrders,

db_hr:
DEPARTMENTS,EMPLOYEES,EMPSALARY,EMP_DEPT,JOBS,JOB_HISTORY,LOCATIONS,

db_learner:
BankAccounts,PETRESCUE,PETSALE,ShoeShop,

db_little_lemon:
Bookings,MenuItems,Menus,Orders,

db_meta:
Clients,ClientsCopy,ClientsCopyTwo,ClientsTest3,Orders,customers,devices,employee_orders,employees_tbl,feedback,orders_tbl,tbl_Orders,tbl_address,tbl_bookings,tbl_contact,tbl_customers,tbl_employees,tbl_invoice,tbl_stock,tbl_students,

db_meta_advanced_mysql:
customers,tbl_customers,tbl_notifications,tbl_orders,tbl_products,

db_meta_dsm_mysql:
BookingsView,bookings_tbl,orders

In [26]:
create_user_query = """CREATE USER 'amblessed'@'localhost' IDENTIFIED WITH caching_sha2_password BY 'WSLuser##++24';"""
grant_user_query = """GRANT ALL PRIVILEGES ON *.* TO 'amblessed'@'localhost' WITH GRANT OPTION;"""

cursor.execute(create_user_query)
cursor.execute(grant_user_query)

In [28]:
cursor.execute("mysql --version")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'mysql --version' at line 1

In [ ]:
# Execute the mysqldump command
backup_file = "backup.sql"
pswd = getpass.getpass('Password:')

command = f"echo {pswd} | sudo -S mysqldump -h{dbconfig['host']} -u{dbconfig['user']} -p{dbconfig['password']} -P{dbconfig['port']} {database_name} > {backup_file}"
process = subprocess.run(command, shell=True)

# Print out status messages based on the return code
if process.returncode == 0:
    print("Database backup completed successfully.")
else:
    print(f"Database backup failed with return code {process.returncode}.")